In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-27T08:25:10.364258-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%watermark --iversions

optuna : 3.1.1
xgboost: 1.7.5
pandas : 1.5.3
dask   : 2023.3.2
logging: 0.5.1.2
numpy  : 1.23.5



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA H100 80GB HBM3 (UUID: GPU-5c583ee7-8fb1-b26a-4bea-16e45d984a32)
GPU 1: NVIDIA H100 80GB HBM3 (UUID: GPU-a6ab06f5-a6e2-18c1-9dec-0dcd29f44a46)
GPU 2: NVIDIA H100 80GB HBM3 (UUID: GPU-bb8d5098-3c56-c48d-a0a3-fcdfcec6d3f5)
GPU 3: NVIDIA H100 80GB HBM3 (UUID: GPU-cdbe686b-1611-999b-8e8d-a3c5f35b40c4)
GPU 4: NVIDIA H100 80GB HBM3 (UUID: GPU-0df1cef0-fc95-cc88-b5f4-239889b3acba)
GPU 5: NVIDIA H100 80GB HBM3 (UUID: GPU-9dca657c-dbe4-08b7-fe7c-5653f183f0b6)
GPU 6: NVIDIA H100 80GB HBM3 (UUID: GPU-33389782-e2ad-5022-997d-cf470313879c)
GPU 7: NVIDIA H100 80GB HBM3 (UUID: GPU-926eaa05-87fb-35e0-0a98-adb2ad41d0be)


In [6]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-05-27 08:25:17,689 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-27 08:25:17,689 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-27 08:25:17,690 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-27

In [7]:
!nvidia-smi

Sat May 27 08:25:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 80G...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   33C    P0   110W / 700W |    525MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA H100 80G...  On   | 00000000:43:00.0 Off |                    0 |
| N/A   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_l.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_l.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train = pd.read_csv('../input/train.csv.zip')

shift = 200

target0 = train['loss'].values
target = np.log(target0+shift)

In [10]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con =2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 


def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [17]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        #'objective': 'reg:squarederror', 
        'base_score':7.76,
        'tree_method':'gpu_hist',  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_boost_round=num_round, obj=logregobj)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = np.exp(predictions) - shift
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    mae = mean_absolute_error(target0, train_oof)
    
    return mae

In [14]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_l_5.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_l_5.db", study_name="five_fold_optuna_xgb_l_5", direction='minimize')

In [15]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/tmp/ipykernel_3793357/2813017132.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_3793357/2813017132.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_3793357/2813017132.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_3793357/2813017132.py:14: FutureWarnin

CPU times: user 41.7 s, sys: 7.96 s, total: 49.7 s
Wall time: 1min 41s


In [16]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1141.127964,2.555825,0.790763,rmse,0.697340,3.357601,0.048073,25,178,0.536325,COMPLETE
1,1,1408.976605,0.002868,0.578181,rmse,0.055715,0.069765,0.001670,3,197,0.789362,COMPLETE
2,2,1235.616349,0.001012,0.354681,rmse,0.307306,0.003165,0.002221,9,36,0.967598,COMPLETE


In [18]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_5.csv', index=False)
df

/tmp/ipykernel_3793357/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_3793357/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_3793357/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_3793357/481481954.py:14: FutureWarning: s

CPU times: user 1min 2s, sys: 12.8 s, total: 1min 15s
Wall time: 3min 20s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1141.127964,2.555825,0.790763,rmse,0.697340,3.357601,0.048073,25,178,0.536325,COMPLETE
1,1,1408.976605,0.002868,0.578181,rmse,0.055715,0.069765,0.001670,3,197,0.789362,COMPLETE
2,2,1235.616349,0.001012,0.354681,rmse,0.307306,0.003165,0.002221,9,36,0.967598,COMPLETE
3,3,1359.737826,5.183680,0.731091,rmse,1.614960,7.712550,0.001167,17,192,0.523368,COMPLETE
4,4,1162.749121,0.036856,0.941004,rmse,4.095969,0.645493,0.016375,25,229,0.700136,COMPLETE
5,5,1139.003557,1.008101,0.669285,rmse,0.255288,2.106221,0.011381,19,274,0.835461,COMPLETE
6,6,1140.994493,0.353164,0.936617,rmse,0.001349,0.082008,0.009765,14,157,0.585679,COMPLETE
7,7,1144.967745,0.003452,0.904177,rmse,0.002806,6.526025,0.026503,13,273,0.445662,COMPLETE


In [19]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_5.csv', index=False)
df.head()

/tmp/ipykernel_3793357/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_3793357/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_3793357/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_3793357/481481954.py:14: FutureWarning: s

CPU times: user 8min 5s, sys: 5min 58s, total: 14min 3s
Wall time: 1h 40min 37s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1141.127964,2.555825,0.790763,rmse,0.697340,3.357601,0.048073,25,178,0.536325,COMPLETE
1,1,1408.976605,0.002868,0.578181,rmse,0.055715,0.069765,0.001670,3,197,0.789362,COMPLETE
2,2,1235.616349,0.001012,0.354681,rmse,0.307306,0.003165,0.002221,9,36,0.967598,COMPLETE
3,3,1359.737826,5.183680,0.731091,rmse,1.614960,7.712550,0.001167,17,192,0.523368,COMPLETE
4,4,1162.749121,0.036856,0.941004,rmse,4.095969,0.645493,0.016375,25,229,0.700136,COMPLETE


In [16]:
df

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1153.904710,0.373965,0.980291,rmse,2.497604,0.014629,0.007077,21,103,0.583175,COMPLETE
1,1,1152.803715,0.004224,0.856359,rmse,0.004733,3.143904,0.041572,11,39,0.653761,COMPLETE
2,2,1152.649488,1.043722,0.903962,rmse,0.709577,3.394003,0.012027,6,49,0.431573,COMPLETE
3,3,1200.088834,0.056953,0.339802,rmse,0.002243,0.008525,0.073172,16,45,0.594994,COMPLETE
4,4,1255.695543,0.246479,0.914929,rmse,0.445993,0.173519,0.002016,25,255,0.547362,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
103,103,1135.769548,3.706865,0.433950,rmse,0.003413,0.313685,0.017379,17,216,0.944976,COMPLETE
104,104,1144.954486,2.962983,0.489431,rmse,0.001621,0.438582,0.010861,15,5,0.989317,COMPLETE
105,105,1135.953851,1.841624,0.512196,rmse,0.003137,0.219404,0.014701,16,193,0.970973,COMPLETE
106,106,1138.415958,4.095898,0.458030,rmse,0.001146,0.372069,0.012790,12,207,0.934205,COMPLETE


In [17]:
df.value.min()

1135.212585472503